# Conversation with AI - Chatbot

In [1]:
# Import libraries
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

import datetime
print(f"This notebook was last run at: {datetime.datetime.now() : %d/%m/%y %H:%M:%S.%f }")

This notebook was last run at:  25/09/25 23:24:05.212799 


## Load environment Variables and models

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your_key_if_not_using_env')
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = "llama3.2:latest"
MODEL_DEEPSEEK = "deepseek-r1:1.5b"

openai = OpenAI()

## Structure of prompt messages to OpenAI:

In [4]:
system_message = "You are a helpful and intelligent assistant."
message = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Hello, world!"},
]

## Create a chat function

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content":system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message}) # Make a recursion until the conversation is off
    
    print("History is:")
    print(history)
    
    print("And message is:")
    print(messages)
    
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)
    response = []
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response.append(chunk.choices[0].delta.content)
            yield "".join(response)


In [12]:
# Force darkmode
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [15]:
gr.ChatInterface(fn=chat,js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And message is:
[{'role': 'system', 'content': 'You are a helpful and intelligent assistant.'}, {'role': 'user', 'content': 'Hello there'}]
History is:
[['Hello there', 'Hello! How can I assist you today?']]
And message is:
[{'role': 'system', 'content': 'You are a helpful and intelligent assistant.'}, {'role': 'user', 'content': 'Hello there'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': "I want to find an scholarship to Lomonosov Univerity. What're the conditions that I need?"}]


In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

History is:
[['Hello there', 'Hello! How can I assist you today?'], ["I want to find an scholarship to Lomonosov Univerity. What're the conditions that I need?", "Lomonosov Moscow State University (MSU) offers various scholarships for both domestic and international students. The requirements and conditions can vary depending on the type of scholarship you are applying for. Here are some general guidelines:\n\n1. **Academic Excellence**: Most scholarships require a strong academic record. This may include grades, standardized test scores, and relevant qualifications.\n\n2. **Language Proficiency**: If you’re applying for courses taught in Russian, you may need to demonstrate proficiency in the language (such as passing a language test). For programs in English, you may need to prove your proficiency through tests like TOEFL or IELTS.\n\n3. **Extracurricular Activities**: In some cases, scholarships may consider your involvement in extracurricular activities, volunteer work, or leadersh

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [22]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"


gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [18]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [19]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [28]:
def chat_ML(message, history):
    base_system_message = """
    Bạn là một trợ lý AI chuyên nghiệp, chuyên hỗ trợ học Machine Learning (ML).
    - Giải thích khái niệm một cách dễ hiểu, có cấu trúc: (khái niệm → giải thích → ví dụ).
    - Có thể minh họa bằng Python (scikit-learn, numpy, pytorch).
    - Giữ giọng văn thân thiện, khuyến khích người học.
    """

    off_topic_keywords = ["phim", "bóng đá", "chính trị", "nấu ăn", "game", "âm nhạc"]

    relevant_system_message = base_system_message
    if any(keyword.lower() in message.lower() for keyword in off_topic_keywords):
        relevant_system_message += """
        Lưu ý: Nếu người dùng hỏi ngoài phạm vi Machine Learning,
        hãy trả lời ngắn gọn, lịch sự và nhắc họ quay lại chủ đề ML.
        Ví dụ: "Xin lỗi, tôi chỉ hỗ trợ về Machine Learning. 
        Bạn có muốn hỏi về một chủ đề ML cụ thể không?"
        """
    messages = [{"role": "system", "content": relevant_system_message}] + history
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response += chunk.choices[0].delta.content
            yield history + [
                {"role": "user", "content": message},
                {"role": "assistant", "content": response}
            ]

# Gradio ChatInterface
demo = gr.ChatInterface(
    fn=chat_ML,
    type="messages",
    title="🤖 Chatbot hỗ trợ học Machine Learning",
    description="Đặt câu hỏi về Machine Learning, tôi sẽ giải thích chi tiết và có ví dụ minh họa."
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [ ]:
gr.ChatI